Aqui eu vou escrever todo o programa e explicar cada parte :)

Primeiro, vou apenas copiar os blocos de código do projeto do Matheus e escrever o que precisa ser alterado.

Uma coisa que tenho percebido ao redor do código é a falta de funções auxiliares. Eu pessoalmente não gosto de main longa e cheia, então acho que vou ajustar isso depois.

# 1. Crawler:

Aqui serão extraídos os nomes e _abstracts_ dos artigos relacionados aos termos de busca.

In [ ]:
import string, os

from Bio import Entrez
from pathlib import Path
import pubchempy as pcp

Essas são funções auxiliares para o funcionamento do _crawler_.

In [ ]:
def list_from_txt(file_path):
    strings_list = []
    with open(file_path, 'rt', encoding='utf-8') as list_file:
        for line in list_file: strings_list.append(line.rstrip('\n'))
    return strings_list

def search(paper_query):
    final_query = '{} AND English[Language]'.format(paper_query)
    Entrez.email = 'tirs@estudante.ufscar.br'
    handle = Entrez.esearch(db='pubmed',
                            sort='relevance',
                            retmax='999999',
                            retmode='xml',
                            term=final_query)
    found = Entrez.read(handle)
    handle.close()
    return found

def fetch_details(paper_ids):
    ids_string = ','.join(paper_ids)
    Entrez.email = 'tirs@estudante.ufscar.br'
    handle = Entrez.efetch(db='pubmed',
                           retmode='xml',
                           id=ids_string)
    found = Entrez.read(handle)
    handle.close()
    return found

def flat(round_list):
    flat_list = []
    for element in round_list:
        if type(element) is list:
            for item in element: flat_list.append(item)
        else: flat_list.append(element)
    return flat_list

Essa ultima função auxiliar é a que insere aos termos de busca sinônimos de compostos para aumentar o escopo da busca.

Aqui está _hardcoded_ os IDs dos compostos que terão seus sinônimos inclusos. Esses compostos são tratamentos conhecidos para a AML, e foram levantados manualmente por um especialista. Preciso substituir esses compostos manualmente escolhidos, arrumando alguma forma de obter esses IDs automaticamente para uma doença escolhida. Como? Não sei ainda.

In [ ]:
def extend_search():
    compound_ids = [
        14888, 9444, 62770, 2907, 6253, 122640033, 5743, 90480031, 76970819, 636362, 71657455, 9829523, 51082, 5865,
        2723601, 249332, 49846579, 11422859
    ]

    substance_ids = [
        404336834
    ]

    extended = [pcp.Compound.from_cid(compound_ids[0]).synonyms]

    for c in compound_ids[1:]:
        extended.extend(pcp.Compound.from_cid(c).synonyms)

    for s in substance_ids:
        extended.extend(pcp.Substance.from_sid(s).synonyms)

    extended = flat(extended)
    remove_words = []
    for s in extended:
        if '[as the base]' in s or '[Poison]' in s or '[ISO]' in s or '[INN]' in s or 'USP/JAN' in s or '(JAN' in s or '[JAN' in s or 'Latin' in s or 'Spanish' in s or '(TN)' in s or '(INN)' in s or 'USAN' in s or 'JP17/USP' in s or 'Czech' in s or 'German' in s or '[CAS]' in s:
            remove_words.append(s)

    extended = [x for x in extended if x not in remove_words]
    extended = [x for x in extended if len(x) >= 3]
    extended = list(dict.fromkeys(extended))

    useless_strings = list_from_txt('./data/bad_search_strings.txt')
    for w in useless_strings:
        try:
            extended.remove(w)
        except ValueError:
            pass

    remove_words = ['Antibiotic U18496', 'D,L-Cyclophosphamide', 'N,O-propylen-phosphorescence-ester-diamid',
                    'UNII-6UXW23996M component CMSMOCZEIVJLDB-CQSZACIVSA-N']
    for w in remove_words:
        try:
            extended.remove(w)
        except ValueError:
            pass

    return extended

Aqui é o programa principal do _crawler_. Rodar isso aqui demora muito, então não sei ainda como vou testar.

Alterações necessárias:

   1. Termos de busca não são fixos, preciso de uma função auxiliar que retorna termos de busca para a doença de escolha.

   2. Parecido com o 1, mas para os sinônimos.

   3. Não acho que esse arquivo de IDs vai continuar existindo.

In [ ]:
if __name__ == '__main__':
    destination_path = './results/'
    Path('./data/ids.txt').touch(exist_ok=True)
    Path(destination_path).mkdir(parents=True, exist_ok=True)

    # Cria uma lista com todos os termos de busca do arquivo. (não vai mais existir esse arquivo)
    queries = list_from_txt('./data/queries.txt')
    paper_counter = 0

    # Insere os sinônimos dos compostos no fim da lista.
    synonyms = extend_search()
    queries.extend(synonyms)

    # Cria uma lista com os IDs de todos os artigos já obtidos e um conjunto de IDs de artigos obtidos.
    old_papers = list_from_txt('./data/ids.txt')
    ids = set(old_papers)

    # Para cada termo de busca...
    for query in queries[100:]:
        # Normaliza o termo de busca.
        query = query.encode('ascii', 'ignore').decode('ascii')
        print('searching for {}'.format(query))

        # Procura pelo termo de busca, salva os IDs dos artigos encontrados em id_list.
        id_list = list(search(query)['IdList'])
        # Mantém só os que ainda não estão no conjunto de IDs conhecidos, ou seja, os novos.
        id_list = [x for x in id_list if x not in ids]

        # Se não achar nada novo, pode ir para o próximo termo de busca.
        if not id_list:
            print('No new papers found\n')
            continue

        print('{} papers found\n'.format(len(id_list)))
        # Insere os novos IDs no conjunto.
        ids.update(id_list)

        # Pega os detalhes de cada artigo novo.
        papers = fetch_details(id_list)

        # Cria uma pasta com o nome do termo de busca, formatado para poder ser nome de pasta.
        folder_name = query.lower().translate(str.maketrans('', '', string.punctuation)).replace(' ', '_')
        Path(destination_path + '{}'.format(folder_name)).mkdir(parents=True, exist_ok=True)

        # Para cada artigo novo...
        for paper in papers['PubmedArticle']:
            article_year = ''
            article_abstract = ''

            # Se não tiver título, vai para o próximo
            try:
                article_title = paper['MedlineCitation']['Article']['ArticleTitle']
                article_title_filename = article_title.lower().translate(
                    str.maketrans('', '', string.punctuation)).replace(' ', '_')
            except KeyError as e: continue

            # Se não tiver prefácio, vai para o próximo.
            try:
                article_abstract = ' '.join(paper['MedlineCitation']['Article']['Abstract']['AbstractText'])
                article_year = paper['MedlineCitation']['Article']['Journal']['JournalIssue']['PubDate']['Year']
            except KeyError as e:
                if 'Abstract' in e.args: continue
                if 'Year' in e.args:
                    article_year = paper['MedlineCitation']['Article']['Journal']['JournalIssue']['PubDate'][
                                       'MedlineDate'][0:4]

            # Se não tiver ano válido, vai para o próximo.
            if len(article_year) != 4: continue

            # O nome do arquivo vai ser "{ano}_{nome_do_artigo}"...
            filename = '{}_{}'.format(article_year, article_title_filename)
            if len(filename) > 150: filename = filename[0:146]

            # e vai ser escrito naquela pasta criada antes do loop.
            path_name = destination_path + folder_name + '/{}.txt'.format(filename)
            path_name = path_name.encode('ascii', 'ignore').decode('ascii')

            # Escreve o arquivo.
            with open(path_name, "a", encoding='utf-8') as file:
                file.write(article_title + ' ' + article_abstract)

            paper_counter += 1

        # Coloca os IDs dos artigos novos no arquivo.
        with open('./data/ids.txt', 'a+', encoding='utf-8') as file:
            for new_id in id_list: file.write('\n' + str(new_id))

    print('Crawler finished with {} papers collected.'.format(len(old_papers) + paper_counter))

# 2. Agregação dos textos por ano

Após pegar todos os prefácios resultantes das buscas, os textos precisam ser agrupados por ano para realizar aquela análise de quão antes o tratamento foi descoberto.

In [ ]:
if __name__ == '__main__':
    start_year = 1900
    end_year = 2025
    source_path = './results/'
    destination_path = './results_aggregated/'

    # Pega os nomes de todos os arquivos que vieram do crawler.
    filenames = list(map(str, Path(source_path).glob('**/*.txt')))
    os.makedirs(destination_path, exist_ok=True)

    # Para cada ano no intervalo...
    for year in range(start_year, end_year + 1):
        # Pega os nomes dos arquivos de artigos que são do ano atual.
        filenames_current_year = [f for f in filenames if Path(f).stem.startswith(str(year))]

        # Se não tiver nenhum desse ano, vai para o próximo.
        if not filenames_current_year: continue

        print(f'{len(filenames_current_year)} papers from {year}.\n')

        # Pega os prefácios de todos os artigos desse ano.
        abstracts = []
        for fname in filenames:
            with open(fname, encoding='utf-8') as infile:
                abstracts.extend(infile.readlines())

        # Junta tudo em um só arquivo texto.
        output_file = Path(destination_path) / f'results_file_1900_{year}.txt'
        with open(output_file, 'w', encoding='utf-8') as f:
            for fname, abstract in zip(filenames, abstracts):
                f.write(f"{Path(fname).stem[5:]}|{abstract}")

# 1. Crawler:

Aqui serão extraídos os nomes e _abstracts_ dos artigos relacionados aos termos de busca.

In [5]:
import string, os
from turtledemo.penrose import start

from Bio import Entrez
from pathlib import Path
import pubchempy as pcp
from prompt_toolkit.key_binding.bindings.named_commands import end_of_file

Essas são funções auxiliares para o funcionamento do _crawler_.

In [2]:
def list_from_txt(file_path):
    strings_list = []
    with open(file_path, 'rt', encoding='utf-8') as list_file:
        for line in list_file: strings_list.append(line.rstrip('\n'))
    return strings_list

def search(paper_query):
    final_query = '{} AND English[Language]'.format(paper_query)
    Entrez.email = 'tirs@estudante.ufscar.br'
    handle = Entrez.esearch(db='pubmed',
                            sort='relevance',
                            retmax='999999',
                            retmode='xml',
                            term=final_query)
    found = Entrez.read(handle)
    handle.close()
    return found

def fetch_details(paper_ids):
    ids_string = ','.join(paper_ids)
    Entrez.email = 'tirs@estudante.ufscar.br'
    handle = Entrez.efetch(db='pubmed',
                           retmode='xml',
                           id=ids_string)
    found = Entrez.read(handle)
    handle.close()
    return found

def flat(round_list):
    flat_list = []
    for element in round_list:
        if type(element) is list:
            for item in element: flat_list.append(item)
        else: flat_list.append(element)
    return flat_list

Essa ultima função auxiliar é a que insere aos termos de busca sinônimos de compostos para aumentar o escopo da busca.

ALTERAÇÕES NECESSÁRIAS: aqui está _hardcoded_ os IDs dos compostos que terão seus sinônimos inclusos. Esses compostos são tratamentos conhecidos para a AML, e foram levantados manualmente por um especialista.

SOLUÇÕES?: preciso substituir esses compostos manualmente escolhidos, arrumando alguma forma de obter esses IDs automaticamente para uma doença escolhida. Como? Não sei ainda.

In [3]:
def extend_search():
    compound_ids = [
        14888, 9444, 62770, 2907, 6253, 122640033, 5743, 90480031, 76970819, 636362, 71657455, 9829523, 51082, 5865,
        2723601, 249332, 49846579, 11422859
    ]

    substance_ids = [
        404336834
    ]

    extended = [pcp.Compound.from_cid(compound_ids[0]).synonyms]

    for c in compound_ids[1:]:
        extended.extend(pcp.Compound.from_cid(c).synonyms)

    for s in substance_ids:
        extended.extend(pcp.Substance.from_sid(s).synonyms)

    extended = flat(extended)
    remove_words = []
    for s in extended:
        if '[as the base]' in s or '[Poison]' in s or '[ISO]' in s or '[INN]' in s or 'USP/JAN' in s or '(JAN' in s or '[JAN' in s or 'Latin' in s or 'Spanish' in s or '(TN)' in s or '(INN)' in s or 'USAN' in s or 'JP17/USP' in s or 'Czech' in s or 'German' in s or '[CAS]' in s:
            remove_words.append(s)

    extended = [x for x in extended if x not in remove_words]
    extended = [x for x in extended if len(x) >= 3]
    extended = list(dict.fromkeys(extended))

    useless_strings = list_from_txt('./data/bad_search_strings.txt')
    for w in useless_strings:
        try:
            extended.remove(w)
        except ValueError:
            pass

    remove_words = ['Antibiotic U18496', 'D,L-Cyclophosphamide', 'N,O-propylen-phosphorescence-ester-diamid',
                    'UNII-6UXW23996M component CMSMOCZEIVJLDB-CQSZACIVSA-N']
    for w in remove_words:
        try:
            extended.remove(w)
        except ValueError:
            pass

    return extended

Aqui é o programa principal do _crawler_. Rodar isso aqui demora muito, então não sei ainda como vou testar.

ALTERAÇÕES NECESSÁRIAS:

   1. Termos de busca não são fixos, preciso de uma função auxiliar que retorna termos de busca para a doença de escolha.

   2. Parecido com o 1, mas para os sinônimos.

   3. Não acho que esse arquivo de IDs vai continuar existindo.

In [4]:
if __name__ == '__main__':
    destination_path = './results/'
    Path('./data/ids.txt').touch(exist_ok=True)
    Path(destination_path).mkdir(parents=True, exist_ok=True)

    # Cria uma lista com todos os termos de busca do arquivo. (não vai mais existir esse arquivo)
    queries = list_from_txt('./data/queries.txt')
    paper_counter = 0

    # Insere os sinônimos dos compostos no fim da lista.
    synonyms = extend_search()
    queries.extend(synonyms)

    # Cria uma lista com os IDs de todos os artigos já obtidos e um conjunto de IDs de artigos obtidos.
    old_papers = list_from_txt('./data/ids.txt')
    ids = set(old_papers)

    # Para cada termo de busca...
    for query in queries[100:]:
        # Normaliza o termo de busca.
        query = query.encode('ascii', 'ignore').decode('ascii')
        print('searching for {}'.format(query))

        # Procura pelo termo de busca, salva os IDs dos artigos encontrados em id_list.
        id_list = list(search(query)['IdList'])
        # Mantém só os que ainda não estão no conjunto de IDs conhecidos, ou seja, os novos.
        id_list = [x for x in id_list if x not in ids]

        # Se não achar nada novo, pode ir para o próximo termo de busca.
        if not id_list:
            print('No new papers found\n')
            continue

        print('{} papers found\n'.format(len(id_list)))
        # Insere os novos IDs no conjunto.
        ids.update(id_list)

        # Pega os detalhes de cada artigo novo.
        papers = fetch_details(id_list)

        # Cria uma pasta com o nome do termo de busca, formatado para poder ser nome de pasta.
        folder_name = query.lower().translate(str.maketrans('', '', string.punctuation)).replace(' ', '_')
        Path(destination_path + '{}'.format(folder_name)).mkdir(parents=True, exist_ok=True)

        # Para cada artigo novo...
        for paper in papers['PubmedArticle']:
            article_year = ''
            article_abstract = ''

            # Se não tiver título, vai para o próximo
            try:
                article_title = paper['MedlineCitation']['Article']['ArticleTitle']
                article_title_filename = article_title.lower().translate(
                    str.maketrans('', '', string.punctuation)).replace(' ', '_')
            except KeyError as e: continue

            # Se não tiver prefácio, vai para o próximo.
            try:
                article_abstract = ' '.join(paper['MedlineCitation']['Article']['Abstract']['AbstractText'])
                article_year = paper['MedlineCitation']['Article']['Journal']['JournalIssue']['PubDate']['Year']
            except KeyError as e:
                if 'Abstract' in e.args: continue
                if 'Year' in e.args:
                    article_year = paper['MedlineCitation']['Article']['Journal']['JournalIssue']['PubDate'][
                                       'MedlineDate'][0:4]

            # Se não tiver ano válido, vai para o próximo.
            if len(article_year) != 4: continue

            # O nome do arquivo vai ser "{ano}_{nome_do_artigo}"...
            filename = '{}_{}'.format(article_year, article_title_filename)
            if len(filename) > 150: filename = filename[0:146]

            # e vai ser escrito naquela pasta criada antes do loop.
            path_name = destination_path + folder_name + '/{}.txt'.format(filename)
            path_name = path_name.encode('ascii', 'ignore').decode('ascii')

            # Escreve o arquivo.
            with open(path_name, "a", encoding='utf-8') as file:
                file.write(article_title + ' ' + article_abstract)

            paper_counter += 1

        # Coloca os IDs dos artigos novos no arquivo.
        with open('./data/ids.txt', 'a+', encoding='utf-8') as file:
            for new_id in id_list: file.write('\n' + str(new_id))

    print('Crawler finished with {} papers collected.'.format(len(old_papers) + paper_counter))

searching for HY-10586R
No new papers found

searching for s1782
5 papers found

searching for Azacitidine, 5-AzaC;Vidaza;CC-486
No new papers found

searching for NA02947
No new papers found

searching for NSC 103-627
9999 papers found

searching for NCGC00090851-18
No new papers found

searching for NS00008631
No new papers found

searching for EN300-118700
No new papers found

searching for 4-Amino-1-beta-D-ribofuranosyl-D-triazin-2(1H)-one
1984 papers found

searching for BRD-K03406345-001-21-1
No new papers found

searching for Z1515383340
No new papers found

searching for 206-280-2
No new papers found

searching for 5AE
252 papers found

searching for InChI=1/C8H12N4O5/c9-7-10-2-12(8(16)11-7)6-5(15)4(14)3(1-13)17-6/h2-6,13-15H,1H2,(H2,9,11,16)/t3-,4-,5-,6-/m1/s1
No new papers found

searching for 23541-50-6
6870 papers found

searching for DAUNORUBICIN HYDROCHLORIDE
3515 papers found

searching for DAUNORUBICIN HCL
256 papers found

searching for Daunomycin hydrochloride
20 pape

KeyboardInterrupt: 

# 2. Agregação dos textos por ano

Após pegar todos os prefácios resultantes das buscas, os textos precisam ser agrupados por ano para realizar aquela análise de quão antes o tratamento foi descoberto.

In [7]:
if __name__ == '__main__':
    start_year = 1900
    end_year = 2025
    source_path = './results/'
    destination_path = './results_aggregated/'

    # Pega os nomes de todos os arquivos que vieram do crawler.
    filenames = list(map(str, Path(source_path).glob('**/*.txt')))
    os.makedirs(destination_path, exist_ok=True)

    # Para cada ano no intervalo...
    for year in range(start_year, end_year + 1):
        # Pega os nomes dos arquivos de artigos que são do ano atual.
        filenames_current_year = [f for f in filenames if Path(f).stem.startswith(str(year))]

        # Se não tiver nenhum desse ano, vai para o próximo.
        if not filenames_current_year:
            continue

        print(f'{len(filenames_current_year)} papers from {year}.\n')

        # Pega os prefácios de todos os artigos desse ano.
        abstracts = []
        for fname in filenames:
            with open(fname, encoding='utf-8') as infile:
                abstracts.extend(infile.readlines())

        # Junta tudo em um só arquivo texto.
        output_file = Path(destination_path) / f'results_file_1900_{year}.txt'
        with open(output_file, 'w', encoding='utf-8') as f:
            for fname, abstract in zip(filenames, abstracts):
                f.write(f"{Path(fname).stem[5:]}|{abstract}")

2 papers from 1915.

1 papers from 1919.

1 papers from 1921.

1 papers from 1941.

2 papers from 1943.

1 papers from 1945.

1 papers from 1951.

4 papers from 1957.

1 papers from 1958.

1 papers from 1959.

3 papers from 1961.

1 papers from 1962.

4 papers from 1963.

3 papers from 1964.

5 papers from 1965.

6 papers from 1966.

11 papers from 1967.

11 papers from 1968.

20 papers from 1969.

34 papers from 1970.

35 papers from 1971.

42 papers from 1972.

28 papers from 1973.

34 papers from 1974.

465 papers from 1975.

441 papers from 1976.

411 papers from 1977.

440 papers from 1978.

433 papers from 1979.

474 papers from 1980.

451 papers from 1981.

488 papers from 1982.

556 papers from 1983.

607 papers from 1984.

625 papers from 1985.

661 papers from 1986.

653 papers from 1987.

637 papers from 1988.

656 papers from 1989.

754 papers from 1990.

691 papers from 1991.

674 papers from 1992.

684 papers from 1993.

694 papers from 1994.

651 papers from 1995.

737 p

OSError: [Errno 28] No space left on device